# Setup

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

2025-03-09 12:07:42.010120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 12:07:42.010667: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 12:07:42.012664: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 12:07:42.018753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 12:07:42.029560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
with open('../data/normalization_stats.json', 'r') as f:
    stats = json.load(f)

In [3]:
model_load = tf.keras.models.load_model("NN_22-tuner.keras")

# The Earth at different distances

In [4]:
modernwithoutO3 = np.load("../data/tests/modernwithoutO3_10pc_luvoir.npy", allow_pickle=True)
uv = modernwithoutO3[0]["NOISY_ALBEDO"]
vis = modernwithoutO3[1]["NOISY_ALBEDO"]
nir = modernwithoutO3[2]["NOISY_ALBEDO"]

In [5]:
uv_mean, uv_std = stats['inputs']['B-UV']['mean'], stats['inputs']['B-UV']['std']
vis_mean, vis_std = stats['inputs']['B-Vis']['mean'], stats['inputs']['B-Vis']['std']
nir_mean, nir_std = stats['inputs']['B-NIR']['mean'], stats['inputs']['B-NIR']['std']

In [6]:
uv_normalized = (uv - uv_mean) / uv_std
vis_normalized = (vis- vis_mean) / vis_std
nir_normalized = (nir - nir_mean) / nir_std

uv_normalized = uv_normalized.reshape(1, 8, 1)
vis_normalized = vis_normalized.reshape(1, 94, 1)
nir_normalized = nir_normalized.reshape(1, 49, 1)

In [7]:
predictions = model_load.predict([uv_normalized, vis_normalized, nir_normalized])

I0000 00:00:1741532863.665862  994703 service.cc:146] XLA service 0x7f9f580330a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741532863.665893  994703 service.cc:154]   StreamExecutor device (0): Host, Default Version
2025-03-09 12:07:43.678238: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step


I0000 00:00:1741532863.884440  994703 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [8]:
physical_targets = [
    "OBJECT-RADIUS-REL-EARTH",
    "OBJECT-GRAVITY",
    "ATMOSPHERE-TEMPERATURE",
    "ATMOSPHERE-PRESSURE"
]
main_chemical_targets = ['O2', 'O3']
other_chemical_targets = ['CH4', 'CO', 'CO2', 'H2O', 'N2', 'N2O']

def denormalize_physical(predictions, stats, targets):
    """
    Inverts the normalization for physical outputs.
    predictions: np.array of shape (batch, len(targets))
    stats: the loaded JSON stats dictionary
    targets: list of physical target names
    Returns:
        np.array with original scale values.
    """
    original = np.zeros_like(predictions)
    for i, target in enumerate(targets):
        # Get stats: min, max, and best_n
        min_val, max_val, best_n = stats['outputs'][target]['min'], stats['outputs'][target]['max'], stats['outputs'][target]['best_n']
        # Inverse transformation: 
        # prediction = ((val - min) / (max - min))^(1/best_n) 
        # Thus, val = min + (max - min) * (prediction)^(best_n)
        original[:, i] = min_val + (max_val - min_val) * (predictions[:, i] ** best_n)
    return original

def denormalize_chemical(predictions, stats, targets):
    """
    Inverts the transformation for chemical outputs.
    predictions: np.array of shape (batch, len(targets))
    stats: the loaded JSON stats dictionary
    targets: list of chemical target names
    Returns:
        np.array with original scale values.
    """
    original = np.zeros_like(predictions)
    for i, target in enumerate(targets):
        best_n = stats['outputs'][target]['best_n']
        # Inverse transformation: prediction = val^(1/best_n) so val = prediction^(best_n)
        original[:, i] = predictions[:, i] ** best_n
    return original

In [ ]:
# Denormalize physical outputs.
physical_original = denormalize_physical(predictions['physical_output'], stats, physical_targets)

# Denormalize chemical outputs.
main_chem_original = denormalize_chemical(predictions['main_chemical_output'], stats, main_chemical_targets)
other_chem_original = denormalize_chemical(predictions['other_chemical_output'], stats, other_chemical_targets)

print("Physical outputs (original scale):")
for target, value in zip(physical_targets, physical_original[0]):
    print(f"{target}: {value}")

print("\nMain chemical outputs (original scale):")
for target, value in zip(main_chemical_targets, main_chem_original[0]):
    print(f"{target}: {value}")

print("\nOther chemical outputs (original scale):")
for target, value in zip(other_chemical_targets, other_chem_original[0]):
    print(f"{target}: {value}")


# Send the remaining values back to PSG to generate the spectrum again!!!!!

Physical outputs (original scale):
OBJECT-RADIUS-REL-EARTH: 1.1382752656936646
OBJECT-GRAVITY: 11.985319137573242
ATMOSPHERE-TEMPERATURE: 277.9378356933594
ATMOSPHERE-PRESSURE: 1521.1312255859375

Main chemical outputs (original scale):
O2: 0.10215122252702713
O3: 5.897689447920129e-09

Other chemical outputs (original scale):
CH4: 4.5234185108711245e-07
CO: 7.743665264570154e-07
CO2: 4.0777686081128195e-05
H2O: 0.0004499039496295154
N2: 0.8969947695732117
N2O: 4.206489734315255e-08


In [10]:
list(modernwithoutO3[0].items())[4:]

[('OBJECT-RADIUS-REL-EARTH', 0.998886815822894),
 ('OBJECT-GRAVITY', '9.807'),
 ('ATMOSPHERE-TEMPERATURE', '288.2'),
 ('ATMOSPHERE-PRESSURE', '1012.9999999999999'),
 ('CO2', 0.0003830067335417856),
 ('O2', 0.2109312445592439),
 ('H2O', 0.0004620659506901333),
 ('CO', 3.209538395756897e-06),
 ('O3', 0.0),
 ('CH4', 1.7157086878024644e-06),
 ('N2O', 3.2295692946869933e-07),
 ('N2', 0.7882167559845431)]